In [102]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import mean

In [16]:
stocks = pd.read_csv("sp_500_stocks.csv")
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [13]:
symbol = "AAPL"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data



{'companyName': 'Apple Inc',
 'marketcap': 2457689821362,
 'week52high': 155.49,
 'week52low': 104.31,
 'week52highSplitAdjustOnly': 152.68,
 'week52lowSplitAdjustOnly': 106.9,
 'week52change': 0.26005244665338,
 'sharesOutstanding': 17240227404,
 'float': 0,
 'avg10Volume': 79291684,
 'avg30Volume': 74701779,
 'day200MovingAvg': 137.51,
 'day50MovingAvg': 147.28,
 'employees': 149158,
 'ttmEPS': 5.3,
 'ttmDividendRate': 0.8517193426834334,
 'dividendYield': 0.005880633213553647,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-01',
 'nextEarningsDate': '2021-10-14',
 'peRatio': 28.12532733407225,
 'beta': 1.542384590452294,
 'maxChangePercent': 55.96803382955541,
 'year5ChangePercent': 4.952223096223909,
 'year2ChangePercent': 1.8446398892301243,
 'year1ChangePercent': 0.281477728713937,
 'ytdChangePercent': 0.11149374753049791,
 'month6ChangePercent': 0.13890051193735434,
 'month3ChangePercent': 0.18544398724854555,
 'month1ChangePercent': 0.03167981135869422,
 'day30ChangePercen

In [14]:
data['year1ChangePercent']

0.281477728713937

### Executing A Batch API Call & Building Our DataFrame


In [118]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]  

In [26]:
symbol_groups = list(chunks(stocks["Ticker"],100))
symbol_strings = []

for i in range(0,len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
    #print(symbol_strings)

my_columns = ['Ticker','Price','One year return','No of shares to buy']

### Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [28]:
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,One year return,No of shares to buy


In [32]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series(
        
                                                 [
                                                     symbol,
                                                     data[symbol]['price'],
                                                     data[symbol]["stats"]["year1ChangePercent"],
                                                     "n/a"
                                                     
                                                 ],index = my_columns),ignore_index = True)
    
    
    
    

In [34]:
final_dataframe

,Ticker,Price,One year return,No of shares to buy
0,A,175.710,0.733871,n/a
1,AAL,18.452,0.49258,n/a
2,AAP,217.830,0.322176,n/a
3,AAPL,151.220,0.283178,n/a
4,ABBV,122.380,0.295521,n/a
...,...,...,...,...
500,YUM,140.070,0.468907,n/a
501,ZBH,148.750,0.057786,n/a
502,ZBRA,589.730,0.993692,n/a
503,ZION,55.339,0.672714,n/a


### Keeping only the top 50 stocks and removing the low momentum stocks

In [52]:
final_dataframe.sort_values('One year return' , ascending = False , inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)


<ipython-input-52-537c2f179b2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('One year return' , ascending = False , inplace = True)


In [53]:
final_dataframe.drop('level_0', axis=1, inplace=True)


In [51]:
final_dataframe

,index,Ticker,Price,One year return,No of shares to buy
0,275,LB,80.290,2.377467,n/a
1,272,KSS,56.920,1.874838,n/a
2,441,TPR,40.030,1.68158,n/a
3,344,NUE,122.010,1.660511,n/a
4,106,COF,175.260,1.638322,n/a
5,129,DFS,126.400,1.557779,n/a
6,251,IT,308.010,1.421845,n/a
7,23,ALB,223.580,1.399177,n/a
8,148,DVN,25.919,1.362328,n/a
9,253,IVZ,24.100,1.342066,n/a


### This function takes in the value of the portfolio size to calculate the position size for each stock

In [59]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input('Enter the size of your portfolio :')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number!\nPlease enter a valid number')
        portfolio_size = input('Enter the size of your portfolio :')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio :10000000
10000000


In [60]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)

200000.0


### Building a momentum strategy based solely on One year return and calculating the number of shares to buy

In [62]:
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'No of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,index,Ticker,Price,One year return,No of shares to buy
0,275,LB,80.290,2.377467,2490
1,272,KSS,56.920,1.874838,3513
2,441,TPR,40.030,1.68158,4996
3,344,NUE,122.010,1.660511,1639
4,106,COF,175.260,1.638322,1141
5,129,DFS,126.400,1.557779,1582
6,251,IT,308.010,1.421845,649
7,23,ALB,223.580,1.399177,894
8,148,DVN,25.919,1.362328,7716
9,253,IVZ,24.100,1.342066,8298


### Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

High-quality momentum stocks show "slow and steady" outperformance over long periods of time
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future.

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

1-month price returns

3-month price returns

6-month price returns

1-year price returns

The abbreviation hqm stands for high-quality momentum.

In [124]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'One-Year price return',
    'One-Year return percentile',
    'Six-month price return',
    'Six-month return percentile',
    'Three-month price return',
    'Three-month return percentile',
    'One-month price return',
    'One-month return percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(pd.Series([
            
            symbol,
            data[symbol]['price'],
            'n/a',
            data[symbol]['stats']['year1ChangePercent'],
            'n/a',
            data[symbol]['stats']['month6ChangePercent'],
            'n/a',
            data[symbol]['stats']['month3ChangePercent'],
            'n/a',
            data[symbol]['stats']['month1ChangePercent'],
            'n/a',
            'n/a'
            
            
        ],index = hqm_columns),ignore_index = True)
   
    

### The following data seemed to have some missing data so this is just to remove those rows

In [125]:
#hqm_dataframe.isnull().values.any()
hqm_dataframe.isnull().sum()
hqm_dataframe.dropna(inplace=True)

Calculating Momentum Percentiles

One-Year Price Return

Six-Month Price Return

Three-Month Price Return

One-Month Price Return

In [126]:
time_periods = [
    'One-Year',
    'Six-month',
    'Three-month',
    'One-month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
#        hqm_dataframe.loc[row,f'{time_period} return percentile'] = 0
         hqm_dataframe.loc[row,f'{time_period} return percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} price return'], hqm_dataframe.loc[row,f'{time_period} price return'])/100



In [127]:
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year price return,One-Year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,A,174.610,n/a,0.752833,0.866267,0.329701,0.888224,0.294438,0.956088,0.138109,0.934132,n/a
1,AAL,19.138,n/a,0.489307,0.664671,0.002732,0.221557,-0.192443,0.041916,-0.013175,0.211577,n/a
2,AAP,213.820,n/a,0.316991,0.437126,0.273827,0.806387,0.07993,0.674651,0.013762,0.337325,n/a
3,AAPL,151.580,n/a,0.27634,0.373253,0.138219,0.520958,0.180395,0.876248,0.03156,0.441118,n/a
4,ABBV,124.190,n/a,0.299046,0.409182,0.154479,0.578842,0.029829,0.530938,0.022161,0.389222,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.620,n/a,0.470032,0.642715,0.282836,0.820359,0.135411,0.792415,0.183071,0.978044,n/a
501,ZBH,147.570,n/a,0.057792,0.127745,-0.079754,0.091816,-0.133075,0.101796,-0.04063,0.131737,n/a
502,ZBRA,586.790,n/a,0.989497,0.946108,0.153237,0.57485,0.165914,0.846307,0.113998,0.884232,n/a
503,ZION,55.200,n/a,0.667336,0.820359,0.042427,0.315369,-0.068785,0.253493,0.126997,0.912176,n/a


### Calculating the HQM Score

We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

In [128]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} return percentile'])
    hqm_dataframe.loc[row,"HQM Score"] = mean(momentum_percentiles)
        
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year price return,One-Year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,A,174.610,n/a,0.752833,0.866267,0.329701,0.888224,0.294438,0.956088,0.138109,0.934132,0.911178
1,AAL,19.138,n/a,0.489307,0.664671,0.002732,0.221557,-0.192443,0.041916,-0.013175,0.211577,0.28493
2,AAP,213.820,n/a,0.316991,0.437126,0.273827,0.806387,0.07993,0.674651,0.013762,0.337325,0.563872
3,AAPL,151.580,n/a,0.27634,0.373253,0.138219,0.520958,0.180395,0.876248,0.03156,0.441118,0.552894
4,ABBV,124.190,n/a,0.299046,0.409182,0.154479,0.578842,0.029829,0.530938,0.022161,0.389222,0.477046
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.620,n/a,0.470032,0.642715,0.282836,0.820359,0.135411,0.792415,0.183071,0.978044,0.808383
501,ZBH,147.570,n/a,0.057792,0.127745,-0.079754,0.091816,-0.133075,0.101796,-0.04063,0.131737,0.113273
502,ZBRA,586.790,n/a,0.989497,0.946108,0.153237,0.57485,0.165914,0.846307,0.113998,0.884232,0.812874
503,ZION,55.200,n/a,0.667336,0.820359,0.042427,0.315369,-0.068785,0.253493,0.126997,0.912176,0.575349


### Selecting the 50 Best Momentum Stocks
As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [129]:
hqm_dataframe.sort_values("HQM Score",ascending = False,inplace = True)
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year price return,One-Year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
251,IT,307.50,n/a,1.392075,0.988024,0.645969,0.994012,0.348237,0.976048,0.19526,0.98004,0.984531
195,FTNT,306.49,n/a,1.287047,0.978044,0.721398,0.996008,0.44528,0.996008,0.153773,0.954092,0.981038
23,ALB,223.05,n/a,1.386314,0.986028,0.378597,0.94012,0.367532,0.98004,0.199127,0.984032,0.972555
344,NUE,120.96,n/a,1.652883,0.994012,1.061755,1.0,0.168849,0.848303,0.311168,1.0,0.960579
78,CARR,57.62,n/a,0.915148,0.926148,0.498867,0.99002,0.283135,0.9501,0.148651,0.946108,0.953094
...,...,...,...,...,...,...,...,...,...,...,...,...
107,COG,15.00,n/a,-0.267291,0.005988,-0.211574,0.01996,-0.186505,0.045908,-0.062292,0.085828,0.039421
42,APA,16.81,n/a,-0.185618,0.01996,-0.178711,0.021956,-0.235128,0.011976,-0.071705,0.063872,0.029441
189,FMC,90.71,n/a,-0.17481,0.021956,-0.152267,0.041916,-0.242217,0.00998,-0.141565,0.00998,0.020958
194,FTI,6.48,n/a,-0.240083,0.011976,-0.166625,0.031936,-0.260846,0.007984,-0.15709,0.005988,0.014471


In [130]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index()

,index,Ticker,Price,Number of shares to buy,One-Year price return,One-Year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
0,251,IT,307.500,n/a,1.392075,0.988024,0.645969,0.994012,0.348237,0.976048,0.19526,0.98004,0.984531
1,195,FTNT,306.490,n/a,1.287047,0.978044,0.721398,0.996008,0.44528,0.996008,0.153773,0.954092,0.981038
2,23,ALB,223.050,n/a,1.386314,0.986028,0.378597,0.94012,0.367532,0.98004,0.199127,0.984032,0.972555
3,344,NUE,120.960,n/a,1.652883,0.994012,1.061755,1.0,0.168849,0.848303,0.311168,1.0,0.960579
4,78,CARR,57.620,n/a,0.915148,0.926148,0.498867,0.99002,0.283135,0.9501,0.148651,0.946108,0.953094
5,490,WST,458.960,n/a,0.653942,0.812375,0.603089,0.992016,0.390399,0.988024,0.212643,0.992016,0.946108
6,283,LLY,277.490,n/a,0.828048,0.896208,0.366325,0.926148,0.382833,0.986028,0.165678,0.966068,0.943613
7,477,WAT,402.870,n/a,0.908833,0.922156,0.457489,0.978044,0.326528,0.968064,0.119399,0.898204,0.941617
8,216,HCA,254.590,n/a,0.867004,0.91018,0.415934,0.962076,0.219166,0.916168,0.134031,0.924152,0.928144
9,275,LB,80.650,n/a,2.384322,1.0,0.825061,0.998004,0.222068,0.918164,0.082522,0.758483,0.918663


In [131]:
portfolio_input()

Enter the size of your portfolio :1000000


In [132]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
print(position_size)

20000.0


### Calculating the number of shares to buy based on position size

In [133]:
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row,'Number of shares to buy'] = math.floor(position_size/hqm_dataframe.loc[row,'Price'])
hqm_dataframe   

C:\Users\Acer\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of shares to buy,One-Year price return,One-Year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM Score
251,IT,307.500,65,1.392075,0.988024,0.645969,0.994012,0.348237,0.976048,0.19526,0.98004,0.984531
195,FTNT,306.490,65,1.287047,0.978044,0.721398,0.996008,0.44528,0.996008,0.153773,0.954092,0.981038
23,ALB,223.050,89,1.386314,0.986028,0.378597,0.94012,0.367532,0.98004,0.199127,0.984032,0.972555
344,NUE,120.960,165,1.652883,0.994012,1.061755,1.0,0.168849,0.848303,0.311168,1.0,0.960579
78,CARR,57.620,347,0.915148,0.926148,0.498867,0.99002,0.283135,0.9501,0.148651,0.946108,0.953094
490,WST,458.960,43,0.653942,0.812375,0.603089,0.992016,0.390399,0.988024,0.212643,0.992016,0.946108
283,LLY,277.490,72,0.828048,0.896208,0.366325,0.926148,0.382833,0.986028,0.165678,0.966068,0.943613
477,WAT,402.870,49,0.908833,0.922156,0.457489,0.978044,0.326528,0.968064,0.119399,0.898204,0.941617
216,HCA,254.590,78,0.867004,0.91018,0.415934,0.962076,0.219166,0.916168,0.134031,0.924152,0.928144
275,LB,80.650,247,2.384322,1.0,0.825061,0.998004,0.222068,0.918164,0.082522,0.758483,0.918663


### Formatting Our Output to Excel

In [146]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer , sheet_name = 'Momentum Strategy' , index = False)


In [147]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [148]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of shares to buy', integer_template],
                    'D': ['One-Year price return', percent_template],
                    'E': ['One-Year return percentile', percent_template],
                    'F': ['Six-month price return', percent_template],
                    'G': ['Six-month return percentile', percent_template],
                    'H': ['Three-month price return', percent_template],
                    'I': ['Three-month return percentile', percent_template],
                    'J': ['One-month price return', percent_template],
                    'K': ['One-month return percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [149]:
writer.save()